In [1]:
import pandas as pd
import os
import glob
import numpy as np

#  Functions

In [2]:
def compare(df, comp1=None, comp2=None, comp3=None):
    """Compares two columns"""
    if df[comp1] == df[comp2]:
        return "Match"
    else:
        return "No Match"

# variables

In [3]:
excel = 'xlsx'

# Smartsheets

In [4]:
# BOP_North Div 2021 Debris Removal Tracker
path1 = r"C:\Users\jacque.trahan\Downloads\BOP_North Div 2021 Debris Removal Tracker*"
load1 = glob.glob(path1 + excel)

df1 = pd.read_excel(max(load1, key=os.path.getctime))

In [5]:
# read in South northern Smart sheeets tracker
path_s = r"C:\Users\jacque.trahan\Downloads\ANVIL_North Div 2021 Debris Removal Tracker*"
load_it_s = glob.glob(path_s + excel)

df_s = pd.read_excel(max(load_it_s, key=os.path.getctime), parse_dates=True)

### Concat Northern and south cental files

In [6]:
df_both = pd.concat([df1, df_s])

In [7]:
smartsheets = df_both.copy()

# RTR for vehicle removal

# Get 71A tickets but only if the disposal site is not blank

### read in disposail site dtat base from hProjectDataExport

In [8]:
# this is the normal 71A RTR unit rate tickets from hProjectDataExport
path3 = r"C:\Users\jacque.trahan\Downloads\hProjectDataExport*"
load3 = glob.glob(path3+excel)
df3 = pd.read_excel(max(load3, key=os.path.getctime), header=2)

In [9]:
RTR_71 = df3.copy()
RTR_71 = RTR_71.copy()
RTR_71 = RTR_71[['Zone', 'Service Code', 'Quan.','Disposal Site']]

#format service code and conditon
RTR_71['Service Code'] = RTR_71['Service Code'].str.strip()
RTR_71 = RTR_71[(RTR_71['Service Code'].isin(['71A'])) & (RTR_71['Disposal Site'].notna())]

#fix APNs
RTR_71['Zone'] = RTR_71['Zone'].str.replace("-", "")
RTR_71['Zone'] = RTR_71['Zone'].str[:3] + "-" + RTR_71['Zone'].str[3:6] + "-" + RTR_71['Zone'].str[6:] 

# get just 71A tickets as a column
RTR_71_ticks = RTR_71.groupby(['Zone'])[['Quan.']].sum()
RTR_71_ticks.reset_index(inplace=True)
RTR_71_ticks.columns = ['Zone Name','71A TICKETS COUNTED FOR 5B TICKETS']



### 5A/B Tickets - this is the normal RTR unit rate tickets from aDataManagerTicketExport

In [10]:
# this is the normal RTR unit rate tickets from aDataManagerTicketExport
path2 = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
load2 = glob.glob(path2 + excel)

df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True, na_values=None)

#### Format RTR data

In [11]:
RTR = df2.copy()
RTR = RTR[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2','Ticket Number']]
RTR = RTR.copy()
RTR['Zone Name'] = RTR['Zone Name'].str.replace("-", "")
RTR['Zone Name'] = RTR['Zone Name'].str[:3] + "-" + RTR['Zone Name'].str[3:6] + "-" + RTR['Zone Name'].str[6:]
RTR['Service Code'] = RTR['Service Code'].str.strip()

#### Format RTR service code data

In [12]:
RTR_Vecs = RTR.copy()
RTR_Vecs = RTR_Vecs[(RTR_Vecs['Is Void'] == False) & (RTR_Vecs['Service Code'].isin(['5A','5B']))]
RTR_Vecs = RTR_Vecs[RTR_Vecs['GIS Zone1'].isin(['NORTHERN DIVISION'])]

RTR_Vecs = RTR_Vecs[['Zone Name', 'Unit Count', 'Service Code', 'Ticket Number']]

# Just 5A Tickets

In [13]:
RTR_5a = RTR_Vecs[RTR_Vecs['Service Code'].isin(['5A'])]
#sum up unit counts
RTR_5a_sum = RTR_5a.groupby(['Zone Name'])[['Unit Count']].sum()

#change Column header
RTR_5a_sum.columns = ["5A Totals(VEHICLES ADJUDICATED ON PARCEL)"]
#reset index
RTR_5a_sum.reset_index(inplace=True)

# Just 5B Tickets

In [14]:
RTR_5b = RTR_Vecs[RTR_Vecs['Service Code'].isin(['5B'])]

#sum up unit counts
RTR_5b_sum = RTR_5b.groupby(['Zone Name'])[['Unit Count']].sum()

# Change column header
RTR_5b_sum.columns = ['5B Totals(VEHICLES ADJUDICATED OFF-SITE)']
#reset index
RTR_5b_sum.reset_index(inplace=True)

# Merge 5A and 5B columns

In [15]:
RTR_Nums = RTR_5a_sum.merge(RTR_5b_sum,
                           how='outer',
                           left_on='Zone Name',
                           right_on='Zone Name')

# merge Unit rate RTR and disposal 71A RTR 

In [16]:
RTR_final = RTR_Nums.merge(RTR_71_ticks,
                          how='outer',
                          left_on='Zone Name',
                          right_on='Zone Name')

### Merge smartsheets and rtr total vehicles
Overview:

* 5B tickets (VEHICLES ADJUDICATED OFF-SITE) - same as 5A kept here **just for reference** (nothing is counted off this)


*  5A tickets (VEHICLES ADJUDICATED ON PARCEL) - are counted from: aDataManagerTicketExport

* 71A tickets (in place for 5B Tickets) - are counted from hProjectDataExport but only if a disposal site was named

* NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS = Smart Sheets Total Vehicle Counts

* TOTAL VEHICLES REMOVED ACCORDING TO RTR TICKETS = 5A + 71A Tickets 

* Matches Removal Ticket Counts - Compares Smart Sheets Vehicle removal counts to total RTR ticket counts(5A + 71A tickets)

* TOTAL RTR TICKETS MINUS SMARTSHEET COUNTS = Total Smart sheet vehicle counts - Total Vehicles Removed According to RTR Tickets (5A + 71A tickets)

In [17]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [18]:
RTR_vec_final = smartsheets.merge(RTR_final,
                             left_on='APN_ROW SEGMENT',
                             right_on='Zone Name',
                             how='left')

In [19]:
RTR_vec_final.rename(columns={'NUMBER OF VEHICLES REMOVED': 'NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS'},
                     inplace=True)

In [20]:
RTR_vec_final =RTR_vec_final[['APN_ROW SEGMENT','STREET #','STREET NAME','STRUCTURAL STATUS','COUNTY',
                           'TOTAL NUMBER OF VEHICLES','5A Totals(VEHICLES ADJUDICATED ON PARCEL)',
                              '5B Totals(VEHICLES ADJUDICATED OFF-SITE)','71A TICKETS COUNTED FOR 5B TICKETS',
                              'NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS']]

In [21]:
RTR_vec_final = RTR_vec_final.copy()
RTR_vec_final.fillna(0, inplace=True)

In [22]:
RTR_vec_final['Total Vehicles Removed According to RTR Tickets'] = (RTR_vec_final['5A Totals(VEHICLES ADJUDICATED ON PARCEL)'] + RTR_vec_final['71A TICKETS COUNTED FOR 5B TICKETS'])

In [23]:
RTR_vec_final['Total RTR tickets minus Smartsheet Counts'] = RTR_vec_final['TOTAL NUMBER OF VEHICLES'] - RTR_vec_final['Total Vehicles Removed According to RTR Tickets']

In [24]:
RTR_vec_final.columns = map(str.upper, RTR_vec_final.columns)

In [25]:
RTR_vec_final.insert(RTR_vec_final.columns.get_loc('TOTAL VEHICLES REMOVED ACCORDING TO RTR TICKETS')+1, "SMART SHEETS COMPARED TO RTR",
                    RTR_vec_final.apply(compare, comp1='TOTAL VEHICLES REMOVED ACCORDING TO RTR TICKETS',
                                        comp2='NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS', axis=1))

In [26]:
RTR_vec_final.set_index('APN_ROW SEGMENT', inplace=True)
RTR_vec_final.to_excel('Vehicle Counts Northern Division.xlsx')

# Contarctor Checks(ECC)

In [27]:
# ecc = pd.read_excel(r"C:\Users\jacque.trahan\Downloads\ECC vehicle aspects.xlsx", sheet_name=1)

In [28]:
# ECC_merge = ecc.merge(RTR_vec_final,
#                      how='left',
#                      left_on='APNs',
#                      right_on='APN_ROW SEGMENT')

In [29]:
# ECC_merge.to_excel('Get APNs.xlsx')